# Gemini for Face Exercise Proficiency Classification

Facial exercises have gained popularity for their potential benefits in improving muscle tone, reducing signs of aging and enhancing overall facial symmetry, being fundamental for speech therapy treatment. However, assessing proficiency in these exercises remains a challenge, as effectiveness depends on precise movements and muscle engagement. This notebook explores the application of Gemini, a cutting-edge AI model, for classifying face exercise proficiency without the need for acquiring labeled datasets or training any custom model.

In [32]:
import os
from google import genai

from dotenv import load_dotenv
from google.genai import types

import time
from google.genai import errors

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

client = genai.Client(api_key=GEMINI_API_KEY)

Here is a sample of how to call the Gemini API. In this experiment, we will focus on Gemini 2.0 Flash, which is a multimodal model capable of processing both text and images and delivering high-quality results. Google's API provides a free tier with a limited number of requests, making it accessible for experimentation.

In [2]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or decisions.



---

Prompts play an important role in guiding the model's responses. The following prompt is designed to elicit a classification of proficiency levels based on the provided image. Each exercise should be associated with a  prompt that includes a breaf description of the exercise and a request for proficiency classification. The prompt is structured to ensure clarity and specificity, allowing the model to understand the task at hand. We believe that the prompt should be clear and concise, providing the model with the necessary context to generate accurate and relevant responses.

In [ ]:
# system_prompt = """"
# Você é um modelo de inteligência artificial especializado na análise de exercícios motores faciais a partir de vídeos curtos. Seu objetivo é avaliar a corretude da execução dos movimentos com base em critérios biomecânicos, precisão da execução e alinhamento com as instruções propostas para cada exercício.
# """

### Prediction with a wrong exercise execution

In [8]:
# Only for videos of size <20Mb
video_file_name = "ex1_errado.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.0-flash',
    # config=types.GenerateContentConfig(
    #     system_instruction=system_prompt,
    # ),
    contents=types.Content(
        parts=[
            types.Part(text='Esse exercício foi performado corretamente, parcialmente ou incorretamente? Descrição: exercício de lateralização de língua. O paciente deve projetar a lingua para fora da bcoa e mover ela para os dois lados',),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Com base no vídeo, parece que o exercício de lateralização da língua não foi realizado. Não há movimentos da língua visíveis no vídeo. Portanto, pode-se dizer que o exercício foi executado incorretamente ou não foi realizado.


### Prediction with a correct exercise execution

It is also possible to ask it to return only a single class or any sort of structured output. In this case, we will ask it to return the class name.

In [9]:
# Only for videos of size <20Mb
video_file_name = "ex1_certo.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.0-flash',
    # config=types.GenerateContentConfig(
    #     system_instruction=system_prompt,
    # ),
    contents=types.Content(
        parts=[
            types.Part(text='Esse exercício foi performado corretamente, parcialmente ou incorretamente [Retorne apenas "Acertou", "Parcial" ou "Errou"]? Descrição: exercício de lateralização de língua. O paciente deve projetar a lingua para fora da boca e mover ela obrigatoriamente para os dois lados',),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Acertou


### Prediction with a partial exercise execution

In [6]:
# Only for videos of size <20Mb
video_file_name = "ex1_parcial.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.0-flash',
    # config=types.GenerateContentConfig(
    #     system_instruction=system_prompt,
    # ),
    contents=types.Content(
        parts=[
            types.Part(text='Esse exercício foi performado corretamente, parcialmente ou incorretamente [Retorne apenas "Acertou", "Parcial" ou "Errou"]? Descrição: exercício de lateralização de língua. O paciente deve projetar a lingua para fora da boca e mover ela obrigatoriamente para os dois lados',),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Acertou


It is possible to see that the model was not able to classify correctly the last image, which is a partial execution of the exercise.

### Gemini 1.5 Pro

This model is consedered one of the best for vision tasks. It is possible to see that it is able to classify the images correctly, but it is not able to return a formatted output. The return was no "Partial" as expected, but it depends on the model interpretation of what the prompt is asking for.

In [10]:
# Only for videos of size <20Mb
video_file_name = "ex1_parcial.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    # config=types.GenerateContentConfig(
    #     system_instruction=system_prompt,
    # ),
    contents=types.Content(
        parts=[
            types.Part(text='Esse exercício foi performado corretamente, parcialmente ou incorretamente [Retorne apenas "Acertou", "Parcial" ou "Errou"]? Descrição: exercício de lateralização de língua. O paciente deve projetar a lingua para fora da boca e mover ela obrigatoriamente para os dois lados',),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Errou. O exercício foi realizado apenas para o lado esquerdo.


In [ ]:
# Only for videos of size <20Mb
video_file_name = "ex1_certo.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    # config=types.GenerateContentConfig(
    #     system_instruction=system_prompt,
    # ),
    contents=types.Content(
        parts=[
            types.Part(text='Esse exercício foi performado corretamente, parcialmente ou incorretamente [Retorne apenas "Acertou", "Parcial" ou "Errou"]? Descrição: exercício de lateralização de língua. O paciente deve projetar a lingua para fora da boca e mover ela obrigatoriamente para os dois lados',),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Acertou


In [13]:
# Only for videos of size <20Mb
video_file_name = "ex1_errado.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    # config=types.GenerateContentConfig(
    #     system_instruction=system_prompt,
    # ),
    contents=types.Content(
        parts=[
            types.Part(text='Esse exercício foi performado corretamente, parcialmente ou incorretamente [Retorne apenas "Acertou", "Parcial" ou "Errou"]? Descrição: exercício de lateralização de língua. O paciente deve projetar a lingua para fora da boca e mover ela obrigatoriamente para os dois lados',),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Errou. O paciente não projetou a língua para fora da boca.


All predictions were right. However, it again did not return the expected output format (single word). It may need some kind of post-processing to extract the class name from the response or some aditional promp engineering. 

Confira alguns dos parâmetros de modelo que você pode configurar. As convenções de nomenclatura variam de acordo com a linguagem de programação.

- stopSequences: especifica o conjunto de sequências de caracteres (até 5) que interromperá a geração de saída. Se especificado, a API vai parar na primeira aparição de um stop_sequence. A sequência de paradas não será incluída como parte da resposta.
- temperature: controla a aleatoriedade da saída. Use valores mais altos para respostas mais criativas e valores mais baixos para respostas mais deterministas. Os valores podem variar de [0,0 a 2,0].
- maxOutputTokens: define o número máximo de tokens a serem incluídos em um candidato.
- topP: muda a forma como o modelo seleciona tokens para saída. Os tokens são selecionados do mais para o menos provável até que a soma das probabilidades seja igual ao valor topP. O valor padrão de topP é 0,95.
- topK: muda a forma como o modelo seleciona tokens para saída. Um topK de 1 significa que o token selecionado é o mais provável entre todos os tokens no vocabulário do modelo, enquanto um topK de 3 significa que o próximo token é selecionado entre os três mais prováveis usando a temperatura. Os tokens são filtrados com base em topP, com o token final selecionado usando a amostragem de temperatura.

---

## Protrude and Retract Tongue

In [ ]:
produde_and_retract_tongue_prompt = '''Veja o video da execução do paciente e avalie se o exercício de protrusão e retração de lingua foi performado corretamente, parcialmente ou incorretamente, de acordo com a descrição do exercício fornecida. 

Descrição: O paciente deve projetar a lingua para fora da boca (obrigatoriamente para frente) e depois deve retrair novamente para dentro. 
- Situações em que o paciente não projetou a lingua significativamente para fora ou não retraiu a lingua completamente são consideradas execuções parciais.
- Situações em que não houve projeção para fora ou que ele não retraiu a lingua ao final são consideradas execuções incorretas.
- Qualquer movimento adicional que não seja a projeção e retração da lingua, como a lateralização, mordidas, caretas etc devem ser considerados incorretos.

Retorne APENAS "Acertou", "Parcial" ou "Errou" e um breve comentário sobre o porquê da sua resposta. A estrutura da resposta deve ser a seguinte:
{
    "resposta": "Acertou/Parcial/Errou",
    "comentario": "Seu comentário aqui"
}
'''

In [ ]:
produde_and_retract_tongue_prompt_2 = '''O video anterior é de uma fonoaudióloga realizando o exercócio de protruir e retrair a língua. Com base nessa execução, avalie a performance de um paciente mostrada a seguir.

Descrição: O paciente deve abrir a boca e projetar a lingua para fora da boca (obrigatoriamente para frente) e depois deve retrair novamente para dentro, mantendo a boca aberta. 
- Situações em que não houve projeção para fora ou que ele não retraiu a lingua ao final são consideradas execuções incorretas.
- Qualquer movimento adicional que não seja a projeção e retração da lingua, como a lateralização, mordidas, caretas etc devem ser considerados incorretos.

Retorne APENAS "Acertou", "Parcial" ou "Errou" e um breve comentário sobre o porquê da sua resposta. A estrutura da resposta deve ser a seguinte:
{
    "resposta": "Acertou/Parcial/Errou",
    "comentario": "Seu comentário aqui"
}
'''

In [26]:
fono_video_file_name = "fono/ex2_fono.mp4"
fono_video_bytes = open(fono_video_file_name, 'rb').read()

video_file_name = "ex2_certo_full.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.5-pro-exp-03-25',
    contents=types.Content(
        parts=[
            types.Part(
                inline_data=types.Blob(data=fono_video_bytes, mime_type='video/mp4')
            ),
            types.Part(text=produde_and_retract_tongue_prompt_2,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

```json
{
    "resposta": "Acertou",
    "comentario": "O paciente realizou o movimento de protruir e retrair a língua para frente corretamente, mantendo a boca aberta e sem movimentos adicionais."
}
```


In [4]:
video_file_name = "ex1_certo.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    contents=types.Content(
        parts=[
            types.Part(text=produde_and_retract_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Claro, aqui está a avaliação da execução do exercício:

```json
{
    "resposta": "Errou",
    "comentario": "O paciente não projetou a língua para fora da boca, apenas a lateralizou para fora dos lábios. O movimento deve ser obrigatoriamente para frente."
}
```


10 reps test

In [6]:
video_file_name = "ex1_errado.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    contents=types.Content(
        parts=[
            types.Part(text=produde_and_retract_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Claro! Aqui está a sua resposta:

```json
{
    "resposta": "Errou",
    "comentario": "O paciente não projetou a língua para fora da boca, portanto, o exercício foi executado incorretamente."
}
```


In [5]:
video_file_name = "ex2_certo_full.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    contents=types.Content(
        parts=[
            types.Part(text=produde_and_retract_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Claro! Aqui está a avaliação do exercício de protrusão e retração de língua:

```json
{
    "resposta": "Acertou",
    "comentario": "O paciente projetou a língua para fora da boca para frente e a retraiu completamente para dentro, repetindo o movimento algumas vezes. Não houve movimentos adicionais incorretos."
}
```


---

## Lateralize Tongue

In [ ]:
lateralize_tongue_prompt = '''Veja o video da execução do paciente e avalie se o exercício de lateralização de lingua foi performado corretamente, parcialmente ou incorretamente, de acordo com a descrição do exercício fornecida. 

Descrição: O paciente deve projetar a lingua para fora da boca e depois deve mover o máximo possivel para os dois lados, um de cada vez, e retrair novamente. 
- Situações em que o paciente moveu a lingua apenas para um dos lados ou que não projetou a ligua para fora completamente são consideradas execuções parciais.
- Situações em que não houve projeção para fora ou que ele não moveu a lingua são consideradas execuções incorretas. Situações sem movimento tambem devem ser consideradas incorretas.
- Qualquer movimento adicional que não seja a projeção e lateralização da lingua, como mordidas, caretas etc devem ser considerados incorretor.

Retorne APENAS "Acertou", "Parcial" ou "Errou" e um breve comentário sobre o porquê da sua resposta. A estrutura da resposta deve ser a seguinte:
{
    "resposta": "Acertou/Parcial/Errou",
    "comentario": "Seu comentário aqui"
}
'''

In [27]:
lateralize_tongue_prompt_2 = '''O video anterior é de uma fonoaudióloga realizando o exercócio de lateralizar a língua. Com base nessa execução, avalie a performance de um paciente mostrada a seguir.

Descrição: O paciente deve abrir a boca, projetar a lingua para fora e depois deve mover para os dois lados, um de cada vez. 
- Situações em que não houve projeção para fora ou que ele não moveu a lingua são consideradas execuções incorretas. Situações sem movimento tambem devem ser consideradas incorretas.
- Qualquer movimento adicional que não seja a projeção e lateralização da lingua, como mordidas, caretas etc devem ser considerados incorretor.

Retorne APENAS "Acertou", "Parcial" ou "Errou" e um breve comentário sobre o porquê da sua resposta. A estrutura da resposta deve ser a seguinte:
{
    "resposta": "Acertou/Parcial/Errou",
    "comentario": "Seu comentário aqui"
}
'''

In [ ]:
fono_video_file_name = "fono/ex1_fono.mp4"
fono_video_bytes = open(fono_video_file_name, 'rb').read()

video_file_name = "ex2_certo_full.mp4"
video_bytes = open(video_file_name, 'rb').read()

max_retries = 5
retry_delay = 2
for attempt in range(max_retries):
    try:
        response = client.models.generate_content(
            model='models/gemini-1.5-pro',
            contents=types.Content(
                parts=[
                    types.Part(
                        inline_data=types.Blob(data=fono_video_bytes, mime_type='video/mp4')
                    ),
                    types.Part(text=lateralize_tongue_prompt_2,),
                    types.Part(
                        inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
                    )
                ]
            )
        )
        break
    except errors.APIError as e:
        if e.code == 503:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt + 1} failed with 503 error. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2
            else:
                print("Max retries reached. Server is still unavailable.")
                raise
        else:
            print(f"An unexpected error occurred: {str(e)}")
            raise
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        raise

print(response.text)

Claro! Aqui está a sua resposta:

```json
{
    "resposta": "Errou",
    "comentario": "O paciente não lateralizou a língua como mostrado no vídeo anterior. Ele apenas projetou a língua para fora em todos os movimentos."
}
```


In [10]:
video_file_name = "ex1_errado.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-1.5-pro',
    contents=types.Content(
        parts=[
            types.Part(text=lateralize_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

Claro, aqui está sua avaliação:

```json
{
    "resposta": "Errou",
    "comentario": "O paciente não projetou a língua para fora da boca e nem realizou o movimento de lateralização. A ausência de movimento é considerada uma execução incorreta."
}
```


In [12]:
video_file_name = "ex1_parcial.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.5-pro-exp-03-25',
    contents=types.Content(
        parts=[
            types.Part(text=lateralize_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

```json
{
    "resposta": "Parcial",
    "comentario": "O paciente projetou a língua para fora, mas moveu apenas para um dos lados (o esquerdo) antes de retrair. Para ser considerado correto, deveria ter movido para ambos os lados."
}
```


In [15]:
video_file_name = "ex1_parcial2.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.5-pro-exp-03-25',
    contents=types.Content(
        parts=[
            types.Part(text=lateralize_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

```json
{
    "resposta": "Acertou",
    "comentario": "O paciente projetou a língua para fora da boca e realizou o movimento de lateralização para ambos os lados, buscando a máxima amplitude, conforme descrito no exercício."
}
```


10 reps test

In [16]:
video_file_name = "ex1_certo_full.mp4"
video_bytes = open(video_file_name, 'rb').read()

response = client.models.generate_content(
    model='models/gemini-2.5-pro-exp-03-25',
    contents=types.Content(
        parts=[
            types.Part(text=lateralize_tongue_prompt,),
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            )
        ]
    )
)

print(response.text)

```json
{
    "resposta": "Acertou",
    "comentario": "O paciente realizou a projeção da língua para fora da boca, moveu-a para ambos os lados com amplitude adequada e retraiu-a, repetindo o movimento corretamente sem ações adicionais incorretas."
}
```


---